In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
while "notebooks" in os.getcwd():
    os.chdir("..")

from src.preprocessing.parser import Parser


from typing import Dict